In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import utils 
import torch
from torch import nn


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
subjID = 'K-Reg-S-18'
task = 'one_dot'
df = utils.load_mouseMovement(subjID, task)
df.head()

,trialno,frameCounter,x-shift,y-shift,dot-x,dot-y,group,subjID
0,1.0,1.0,-15.2,24.4,934.096038,527.534657,S,K-Reg-S-18
1,1.0,2.0,-0.8,0.8,933.121802,526.959450,S,K-Reg-S-18
2,1.0,3.0,-0.4,0.0,933.039034,526.568106,S,K-Reg-S-18
3,1.0,4.0,-0.4,0.0,932.952638,526.177548,S,K-Reg-S-18
4,1.0,5.0,-0.4,0.0,932.857495,525.789028,S,K-Reg-S-18


In [4]:
screenSise = utils.getScreenSise(df)
df_ = df[['trialno', 'x-shift', 'y-shift']].copy()
df_['x-shift'] /= screenSise
df_['y-shift'] /= screenSise
df_.head()

,trialno,x-shift,y-shift
0,1.0,-0.008000,0.012842
1,1.0,-0.000421,0.000421
2,1.0,-0.000211,0.000000
3,1.0,-0.000211,0.000000
4,1.0,-0.000211,0.000000


In [5]:
nValidTrials = 6

trials = set(df_['trialno'])
trials_val = np.random.choice(list(trials), nValidTrials)
trial_train = trials.difference(trials_val)

In [6]:
wSize = int(1.2 * 60)
interval = 1

d_train = []
d_val = []
for iTrial in set(df_['trialno']):
    d_ = df_.query(f'trialno == {iTrial}')
    d_ = d_.drop(columns='trialno').values
    d_ = utils.rollingWindow(d_, wSize, interval)
    if iTrial in trial_train:
        d_train.append(d_)
    else:
        d_val.append(d_)

d_train = np.concatenate(d_train, axis=0)
d_val = np.concatenate(d_val, axis=0)


In [7]:
class TrajDataset(torch.utils.data.Dataset):
    def __init__(self, d):
        self.d = d

    def __len__(self):
        return self.d.shape[0]
    
    def __getitem__(self, idx):
        return self.d[idx]

dataset_train = TrajDataset(d_train)
dataset_val = TrajDataset(d_val)


In [8]:
class TrajNet(nn.Module):
    def __init__(self, nHidden=8, nFeature=16, nhead=4, dim_feedforward=64, num_layers=4, dropout=0.1):
        super(TrajNet, self).__init__()
        
        self.nHidden = nHidden
        self.nFeature = nFeature

        # encoder
        self.conv_enc = nn.Conv1d(2, nFeature, 1)
        encoder_ = nn.TransformerEncoderLayer(d_model=nFeature,
                                              nhead=nhead,
                                              dim_feedforward=dim_feedforward,
                                              batch_first=True, 
                                              activation='gelu',
                                              dropout=dropout)
        self.encoder = nn.TransformerEncoder(encoder_, num_layers=num_layers)

        # hidden bottleneck
        self.hidden = nn.Linear(nFeature, nHidden)
        self.memory = nn.Linear(nHidden, nHidden * nFeature)

        # decoder
        self.conv_dec1 = nn.Conv1d(2, nFeature, 1)    
        decoder_ = nn.TransformerDecoderLayer(d_model=nFeature, 
                                              nhead=nhead, 
                                              dim_feedforward=dim_feedforward, 
                                              batch_first=True, 
                                              activation='gelu',
                                              dropout=dropout)
        self.decoder = nn.TransformerDecoder(decoder_, num_layers=num_layers)
        generator_ = nn.TransformerEncoderLayer(d_model=nFeature, 
                                                nhead=nhead, 
                                                dim_feedforward=dim_feedforward, 
                                                batch_first=True, 
                                                activation='gelu',
                                                dropout=dropout)        
        self.generator = nn.TransformerEncoder(generator_, num_layers=num_layers)
        self.conv_dec2 = nn.Conv1d(nFeature, 2, 1)
        
        # output
        # self.output = nn.Linear(nFeature, 2)        

    def forward(self, src, tgt):
        # src.shape = (batch_size, nSrc, nFeature=2)
        # tgt.shape = (batch_size, nTgt, nFeature=2)
        
        # ---------------------------------------------------------------------------- #
        #                                   encoding                                   #
        # ---------------------------------------------------------------------------- #
        nBatch = src.shape[0]
        nTime = src.shape[1]
        x = src
        
        # expend feature dimension
        #   x_expand = (batch, time, nFeature)
        x = x.permute(0, 2, 1)
        x = self.conv_enc(x)
        x = x.permute(0, 2, 1)

        x = self.encoder(x)
        x = torch.tanh(x)

        # extract hidden control units
        #   x.shape = (batch, head)
        x = x[:, 0, :]
        hidden = self.hidden(x)
        y = torch.tanh(hidden)

        # construct output embedding
        memory = self.memory(y)
        memory = memory.reshape(nBatch, self.nHidden, self.nFeature)
        
        # ---------------------------------------------------------------------------- #
        #                                   decoding                                   #
        # ---------------------------------------------------------------------------- #
        # # decode control command
        y = tgt.permute(0, 2, 1)
        y = self.conv_dec1(y)
        y = y.permute(0, 2, 1)
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt.shape[1]).to(device)
        y = self.decoder(y, memory, tgt_mask=tgt_mask)

        # # generate motor sequence
        # #   y.shape = (batch_size, nTime, nFeature=2)
        # src_mask = nn.Transformer.generate_square_subsequent_mask(y.shape[1]).to(device)
        # y = self.generator(y, mask=src_mask)
        y = y.permute(0, 2, 1)
        y = self.conv_dec2(y)
        y = y.permute(0, 2, 1)
        return y
    
    def autoRegressiveDecoding(self, src, nTime):
        tgt = torch.zeros((src.shape[0], 1, 2)).to(device).float()
        for i in range(nTime):
            y = self.forward(src, tgt)[:, -1:, :]
            tgt = torch.concat((tgt, y), dim=1)
        return tgt
    
# model = TrajNet().to(device)
# # y = model(torch.rand(4, 3, 2))
# y = model(torch.rand(4, 3, 2).to(device), torch.rand(4, 4, 2).to(device))
# y.shape


In [9]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
model = TrajNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

In [130]:
model.train()
for epoch in range(30):
    running_loss = 0.0
    for i, batch in enumerate(train_loader):
        optimizer.zero_grad()
        x = batch.to(device).float()
        tgt = torch.roll(x, 1, 1)
        tgt[:, 0, :] = 0
        tgt_ = x
        y = model(x, tgt)
        loss = criterion(y, tgt_)
        loss.backward()
        optimizer.step()        
        running_loss += loss.item()
        # print(f'loss: {loss.item()}')
    print(f'epoch:{epoch}, running_loss: {running_loss/(i+1)}')


epoch:0, running_loss: 3.086700063559152e-05
epoch:1, running_loss: 3.1481736139952414e-05
epoch:2, running_loss: 3.156864272803188e-05
epoch:3, running_loss: 3.075204411513753e-05
epoch:4, running_loss: 3.122811898551329e-05
epoch:5, running_loss: 3.098993471850036e-05
epoch:6, running_loss: 3.13524639759916e-05
epoch:7, running_loss: 3.0865800224476225e-05
epoch:8, running_loss: 3.081824833475601e-05
epoch:9, running_loss: 3.092597904169546e-05
epoch:10, running_loss: 3.2712403227969374e-05
epoch:11, running_loss: 3.096789685552028e-05
epoch:12, running_loss: 3.1752775106087e-05
epoch:13, running_loss: 3.142090553382397e-05
epoch:14, running_loss: 3.1345154817132684e-05
epoch:15, running_loss: 3.0717182883179084e-05
epoch:16, running_loss: 3.126566242048584e-05
epoch:17, running_loss: 3.094824420334111e-05
epoch:18, running_loss: 3.090351125963222e-05
epoch:19, running_loss: 3.244661116489802e-05
epoch:20, running_loss: 3.179961261115956e-05
epoch:21, running_loss: 3.074184024092002e

In [1]:
d = dataset_train[np.random.randint(0, len(dataset_train))]
# d = dataset_val[63]

fig, axs = plt.subplots()
axs.plot(0, 0, 'or')
axs.plot(d[:, 0].cumsum(), d[:, 1].cumsum(), '-')
axs.axis('equal')

model.eval()
src = torch.from_numpy(d).unsqueeze(0).to(device).float()
d_ = model.forward(src, src)
d_ = d_.detach().cpu().numpy()
d_ = d_.squeeze()
axs.plot(d_[:, 0].cumsum(), d_[:, 1].cumsum(), '-')

d_ = model.autoRegressiveDecoding(src, src.shape[1])
d_ = d_.detach().cpu().numpy()
d_ = d_.squeeze()
axs.plot(d_[:, 0].cumsum(), d_[:, 1].cumsum(), '-')
# d_

NameError: name 'dataset_train' is not defined

In [129]:
torch.save(model, 'model_test_local.pt')


In [ ]:
model.train()
for epoch in range(10):
    running_loss = 0.0
    for i, batch in enumerate(train_loader):
        optimizer.zero_grad()
        x = batch.to(device).float()
        y = model(x)
        loss = criterion(y, x)
        loss.backward()
        optimizer.step()        
        running_loss += loss.item()
        # print(f'loss: {loss.item()}')
    print(f'epoch:{epoch}, running_loss: {running_loss/(i+1)}')


AttributeError: 'numpy.ndarray' object has no attribute 'permute'

In [ ]:
d_ = torch.from_numpy(d).unsqueeze(0).to(device).float()
d_ = model.forward(d_)
d_

In [ ]:
d = dataset_train[2000]
src = torch.from_numpy(d).unsqueeze(0).to(device).float()
tgt = torch.zeros((1, 1, 2)).to(device).float()
model.eval()
optimizer.zero_grad()
nTime = int(1.2 * 60)
for i in range(10):
    y = model(src, tgt)[:, -1:, :]
    tgt = torch.concat((tgt, y), dim=1)
tgt = tgt[:, 1:, :]

In [ ]:
fig, axs = plt.subplots()
axs.plot(d[:, 0].cumsum(), d[:, 1].cumsum(), '-')
axs.axis('equal')

d_ = tgt.detach().cpu().numpy()
d_ = d_.squeeze()
axs.plot(d_[:, 0].cumsum(), d_[:, 1].cumsum(), '-')

In [ ]:
d = dataset_train[2000]
d_ = torch.from_numpy(d).unsqueeze(0).to(device).float()
model.eval()
# genTraj = 
i = 0
d_ = model.forward(d_[:, i, :].unsqueeze(1), add_tgt_mask=False)
d_

In [ ]:
a = [1, 2, 3, 4, 5]
a[1]

In [ ]:
d

In [ ]:
decoder_layer = nn.TransformerDecoderLayer(d_model=8, nhead=8)
transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
memory = torch.rand(10, 2, 8)
tgt = torch.rand(4, 2, 8)
transformer_decoder.eval()

In [ ]:
tgt_mask = nn.Transformer.generate_square_subsequent_mask(4)
out = transformer_decoder(tgt, memory, tgt_mask=tgt_mask)
print(out[:, 0, :])

out = transformer_decoder(tgt, memory)
print(out[:, 0, :])

In [ ]:
for i in range(4):
    tgt_mask = nn.Transformer.generate_square_subsequent_mask(i+1)
    if i == 0:
        out = transformer_decoder(tgt[i, :, :].unsqueeze(0), memory, tgt_mask=tgt_mask)
    else:
        out = transformer_decoder(tgt[0:(i+1), :, :], memory, tgt_mask=tgt_mask)
    # out_.append(out)
    print(out[:, 0, :])
    
for i in range(4):
    tgt_mask = nn.Transformer.generate_square_subsequent_mask(i+1)
    if i == 0:
        out = transformer_decoder(tgt[i, :, :].unsqueeze(0), memory, tgt_mask=None)
    else:
        out = transformer_decoder(tgt[0:(i+1), :, :], memory, tgt_mask=None)
    # out_.append(out)
    print(out[:, 0, :])    

In [ ]:
nn.Transformer.generate_square_subsequent_mask(1)

In [ ]:
decoder_layer = nn.TransformerDecoderLayer(d_model=512, nhead=8)
transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
# memory = torch.rand(10, 32, 512)
# tgt = torch.rand(20, 32, 512)


In [ ]:
out = transformer_decoder(tgt, memory)
out

In [ ]:
class TrajNet(nn.Module):
    def __init__(self, nFeature=64, nhead=8, dim_feedforward=128, num_layers=4, dropout=0.1):
        super(TrajNet, self).__init__()

        # encoder
        self.conv_enc = nn.Conv1d(2, nFeature, 1)
        encoder_ = nn.TransformerEncoderLayer(d_model=nFeature,
                                              nhead=nhead,
                                              dim_feedforward=dim_feedforward,
                                              batch_first=True, 
                                              activation='gelu',
                                              dropout=dropout)
        self.encoder = nn.TransformerEncoder(encoder_, num_layers=num_layers)

        # hidden bottleneck
        self.hidden = nn.Linear(nFeature, nFeature)

        # decoder
        self.conv_dec1 = nn.Conv1d(2, nFeature, 1)        
        decoder_ = nn.TransformerEncoderLayer(d_model=nFeature, 
                                              nhead=nhead, 
                                              dim_feedforward=dim_feedforward, 
                                              batch_first=True, 
                                              activation='gelu',
                                              dropout=dropout)
        self.decoder = nn.TransformerEncoder(decoder_, num_layers=num_layers)
        generator_ = nn.TransformerEncoderLayer(d_model=nFeature, 
                                                nhead=nhead, 
                                                dim_feedforward=dim_feedforward, 
                                                batch_first=True, 
                                                activation='gelu',
                                                dropout=dropout)        
        self.generator = nn.TransformerEncoder(generator_, num_layers=num_layers)
        self.conv_dec2 = nn.Conv1d(nFeature, 2, 1)
        
        # output
        # self.output = nn.Linear(nFeature, 2)        

    def forward(self, src):
        # src.shape = (batch_size, nSrc, nFeature=2)
        # tgt.shape = (batch_size, nTgt, nFeature=2)
        
        # ---------------------------------------------------------------------------- #
        #                                   encoding                                   #
        # ---------------------------------------------------------------------------- #
        nTime = src.shape[1]
        x = src
        
        # expend feature dimension
        #   x_expand = (batch, time, nFeature)
        x = x.permute(0, 2, 1)
        x = self.conv_enc(x)
        x = x.permute(0, 2, 1)

        x = self.encoder(x)
        x = nn.functional.gelu(x)

        # extract hidden control units
        #   x.shape = (batch, head)
        x = x[:, 0, :]
        hidden = self.hidden(x)
        y = nn.functional.gelu(hidden)

        # construct output embedding
        y = torch.unsqueeze(y, 1)
        y = y.repeat((1, nTime, 1))
        
        
        # ---------------------------------------------------------------------------- #
        #                                   decoding                                   #
        # ---------------------------------------------------------------------------- #

           
        # decode control command
        y = self.decoder(y)

        # generate motor sequence
        #   y.shape = (batch_size, nTime, nFeature=2)
        src_mask = nn.Transformer.generate_square_subsequent_mask(y.shape[1]).to(device)
        y = self.generator(y, mask=src_mask)
        y = y.permute(0, 2, 1)
        y = self.conv_dec2(y)
        y = y.permute(0, 2, 1)
        return y
    
    # def autoRegressiveDecoding(self, src, nTime):
    #     tgt = torch.zeros((src.shape[0], 1, 2)).to(device).float()
    #     for i in range(nTime):
    #         y = self.forward(src, tgt)[:, -1:, :]
    #         tgt = torch.concat((tgt, y), dim=1)
    #     return tgt[:, 1:, :]
    
# model = TrajNet().to(device)
# model(torch.rand(2, 3, 2).to(device).float())


In [ ]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8, dropout=0.1)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
src = torch.rand(10, 32, 512)


In [ ]:
transformer_encoder.eval()
out = transformer_encoder(src)
out

In [ ]:
# auto-encoder model
class TrajNet(nn.Module):
    def __init__(self, nHidden=64, nFeature=512, nhead=4, dim_feedforward=128, num_layers=4, dropout=0.1):
        super(TrajNet, self).__init__()
        self.nHidden = nHidden
        self.nFeature = nFeature
        
        # encoding
        self.conv_enc = nn.Conv1d(2, nFeature, 1)
        encoder_ = nn.TransformerEncoderLayer(d_model=nFeature,
                                              nhead=nhead,
                                              dim_feedforward=dim_feedforward,
                                              batch_first=True, 
                                              activation='gelu',
                                              dropout=dropout)
        self.encoder = nn.TransformerEncoder(encoder_, num_layers=num_layers)   
        
        # hidden bottleneck
        self.hidden = nn.Linear(nFeature, nHidden)   
        
        # decoding        
        self.linear = nn.Linear(nHidden, 72 * nFeature)
        self.conv_dec1 = nn.Conv1d(nHidden, nFeature, 1)
        decoder_ = nn.TransformerEncoderLayer(d_model=nFeature, 
                                              nhead=nhead, 
                                              dim_feedforward=dim_feedforward, 
                                              batch_first=True, 
                                              activation='gelu',
                                              dropout=dropout)
        self.decoder = nn.TransformerEncoder(decoder_, num_layers=num_layers)
        self.conv_dec2 = nn.Conv1d(nFeature, 2, 1) 
        
    def forward(self, x):
        nBatch = x.shape[0]
        nTime = x.shape[1]
        
        # encoding
        x = x.permute(0, 2, 1)
        x = self.conv_enc(x)
        x = x.permute(0, 2, 1)
        x = self.encoder(x)
        
        # hidden bottleneck
        x = x[:, 0, :]
        hidden = self.hidden(x)
        
        
        # decoding
        y = self.linear(hidden)
        y = y.reshape((nBatch, 72, self.nFeature))
        # y = nn.functional.selu(y)
        # y = torch.ones((nBatch, nTime, self.nHidden)).to(device)
        # y[:, 0, :] = hidden
        # y = y.permute(0, 2, 1)
        # y = self.conv_dec1(y)
        # y = y.permute(0, 2, 1)
        # y = nn.functional.selu(y)
        
        y = self.decoder(y)
        y = y.permute(0, 2, 1)
        y = self.conv_dec2(y)
        y = y.permute(0, 2, 1)
        
        
        return y


# x = torch.rand(3, 72, 2)
# model = TrajNet()
# y = model(x)
# y.shape
# y
